In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pip install wandb

In [ ]:
wandb login

In [ ]:
import torch
from datasets import load_dataset
from IPython.display import Markdown
from peft import LoraConfig, get_peft_model
from transformers import EarlyStoppingCallback, TrainingArguments
from trl import SFTTrainer

In [ ]:
import os
os.environ["HF_TOKEN"] = " "
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["WANDB_API_KEY"] = " "
os.environ["WANDB_PROJECT"] = "raft-QWEN3-14B"
os.environ["WANDB_NAME"] = "RAFT-QWEN3-14B"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Is Bfloat16 avaiable?: {torch.cuda.is_bf16_supported()}")

Is Bfloat16 avaiable?: True


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

qwen_models = [
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",      # Qwen 14B 2x faster
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-0.6B-unsloth-bnb-4bit",
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

<ipython-input-6-53f4ff8e2355>:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
generation_config = {
    "max_new_tokens": 3000,
    "do_sample": True,
    "temperature": 0 + 1e-5,
    "top_k": 20,
    "top_p":0.80,
}

In [ ]:
dataset = load_dataset("DoganK01/RAFT-DATASET-HEALTHCARE", split="train")

In [ ]:
print(dataset[0])

{'id': 'seed_task_0', 'type': 'general', 'question': 'What enzymes degrade glyphosate?', 'context': {'sentences': [['racemosum. Furthermore, results revealed antioxidant activity without any cytotoxicity on the Vero normal cell line. The results of this study warrant further in vivo experiments.Author ContributionsConceptualization, S.S.S. and A.H.H.; methodology, S.S.S. and A.H.H.; validation, S.S.S. and A.H.H.; formal analysis, S.S.S. and A.H.H.; investigation, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H.; software S.S.S. and A.H.H.; resources, S.S.S. and A.H.H.; data curation, S.S.S. and A.H.H.; Funding O.M.A. and K.A.A.-E.; writing\\u2014original draft preparation, S.S.S. and A.H.H.; writing\\u2014review and editing, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H.; visualization, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H. All authors have read and agreed to the published version of the manuscript.FundingThis study was supported by the Taif University Researchers Supporting Project (

In [ ]:
chat = [
    {"role": "user", "content": "You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France?"},
    {"role": "assistant", "content": "In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris"},
    {"role": "user", "content": "What is the capital of Germany?"}
]


formatted_chat = tokenizer.apply_chat_template(conversation=chat, tokenize=False, add_generation_prompt=True, enable_thinking=True)
formatted_chat_token = tokenizer.apply_chat_template(conversation=chat, add_generation_prompt=True, return_tensors="pt")
print(formatted_chat)

<|im_start|>user
You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France?<|im_end|>
<|im_start|>assistant
In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris<|im_end|>
<|im_start|>user
What is the capital of Germany?<|im_end|>
<|im_start|>assistant



In [ ]:
print(f"Vocabulary size of QWEN3: {len(tokenizer.get_vocab()):,}")

Vocabulary size of QWEN3: 151,669


In [ ]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|vision_pad|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [ ]:
input_ids = tokenizer(text=formatted_chat, return_tensors="pt").to(device)
print(input_ids)

{'input_ids': tensor([[151644,    872,    198,   2610,    525,    458,  13833,  33968,     11,
            323,    498,  19032,    448,  68432,    435,  77455,    323,    662,
          98730,  30668,   1087,     13,   3555,    374,    279,   6722,    315,
           9625,     30, 151645,    198, 151644,  77091,    198,    641,    279,
          21889,    315,   3100,    323,   7674,     11,   1380,    279,  10948,
           1345,    482,    594,  21700,   1482,  27455,     11,   1052,  15448,
            264,   3283,    315,   1947,    323,   2948,     13,  11445,   4746,
          33327,    448,    279,  36290,    315,  27597,     11,  54937,   1095,
            304,    279,  42646,    316,    315,    279,   4268,    315,    279,
          94526,     82,     13,  29739,    279,   3283,   1380,    279,   6662,
           9729,  48506,  13352,   7616,     11,  10282,    279,  23594,    315,
            279,  98783,  28556,     11,    323,   1052,    498,   4880,   1477,
            27

In [ ]:
outputs = model.generate(**input_ids, **generation_config)
print(outputs)

tensor([[151644,    872,    198,   2610,    525,    458,  13833,  33968,     11,
            323,    498,  19032,    448,  68432,    435,  77455,    323,    662,
          98730,  30668,   1087,     13,   3555,    374,    279,   6722,    315,
           9625,     30, 151645,    198, 151644,  77091,    198,    641,    279,
          21889,    315,   3100,    323,   7674,     11,   1380,    279,  10948,
           1345,    482,    594,  21700,   1482,  27455,     11,   1052,  15448,
            264,   3283,    315,   1947,    323,   2948,     13,  11445,   4746,
          33327,    448,    279,  36290,    315,  27597,     11,  54937,   1095,
            304,    279,  42646,    316,    315,    279,   4268,    315,    279,
          94526,     82,     13,  29739,    279,   3283,   1380,    279,   6662,
           9729,  48506,  13352,   7616,     11,  10282,    279,  23594,    315,
            279,  98783,  28556,     11,    323,   1052,    498,   4880,   1477,
            279,  43621,    

In [ ]:
tokenizer.batch_decode(outputs)

['<|im_start|>user\nYou are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France?<|im_end|>\n<|im_start|>assistant\nIn the realm of light and culture, where the River Seine\'s gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris<|im_end|>\n<|im_start|>user\nWhat is the capital of Germany?<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, the user asked for the capital of Germany. Let me think about how to approach this. They previously asked about France, and I used a poetic, riddle-like response. Now they want the same style for Germany.\n\nFirst, I need to recall the capital of Germany. It\'s 

In [ ]:
from typing import Dict, List

def format_example(batch: Dict[str, List], return_answer: bool = True) -> Dict[str, List[str]]:
    texts = []

    instruction = (
        "Carefully read and analyze the provided documents to answer the question that follows. "
        "Provide a detailed, step-by-step explanation of your reasoning, demonstrating how you arrived at your"
        " conclusion based on the information given in the documents.\n\n"
    )

    for i in range(len(batch["instruction"])):
        user_input = (
            " ".join(batch["instruction"][i]) if isinstance(batch["instruction"][i], list)
            else batch["instruction"][i]
        )
        chat = [{"role": "user", "content": instruction + user_input}]

        prompt = tokenizer.apply_chat_template(
            conversation=chat,
            tokenize=False,
            add_generation_prompt=True
        )

        if return_answer:
            prompt += f"{batch['cot_answer'][i]}<|im_end|>"

        texts.append(prompt)

    return {"text": texts}


In [ ]:
dataset = dataset.map(lambda batch: format_example(batch, return_answer=True), batched=True)


In [ ]:
print(dataset[0])

{'id': 'seed_task_0', 'type': 'general', 'question': 'What enzymes degrade glyphosate?', 'context': {'sentences': [['racemosum. Furthermore, results revealed antioxidant activity without any cytotoxicity on the Vero normal cell line. The results of this study warrant further in vivo experiments.Author ContributionsConceptualization, S.S.S. and A.H.H.; methodology, S.S.S. and A.H.H.; validation, S.S.S. and A.H.H.; formal analysis, S.S.S. and A.H.H.; investigation, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H.; software S.S.S. and A.H.H.; resources, S.S.S. and A.H.H.; data curation, S.S.S. and A.H.H.; Funding O.M.A. and K.A.A.-E.; writing\\u2014original draft preparation, S.S.S. and A.H.H.; writing\\u2014review and editing, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H.; visualization, S.S.S., O.M.A., A.M.R., K.A.A.-E. and A.H.H. All authors have read and agreed to the published version of the manuscript.FundingThis study was supported by the Taif University Researchers Supporting Project (

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=1399, shuffle=True)
train_data = train_test_split["train"].shuffle()
val_data = train_test_split["test"].shuffle()
print(f"Train size: {len(train_data)}", f"Test size: {len(val_data)}")

Train size: 3148 Test size: 350


In [ ]:
torch.manual_seed(1402)
sample = train_data[torch.randint(low=0, high=len(train_data), size=(1,)).item()]

In [ ]:
def format_example_without_batch(example, return_answer=True):

    instruction = "Carefully read and analyze the provided documents to answer the question that follows. " \
    "Provide a detailed, step-by-step explanation of your reasoning, demonstrating how you arrived at your" \
    " conclusion based on the information given in the documents.\n\n"

    chat = [{"role": "user", "content": instruction + example["instruction"]}]

    prompt = tokenizer.apply_chat_template(
        conversation=chat,
        tokenize=False,
        add_generation_prompt=True
    )

    if return_answer:
        prompt += f" {example['cot_answer']}</s>"

    return prompt

In [ ]:
print(format_example_without_batch(example=sample, return_answer=True)[:1200])
print("...")
print(format_example_without_batch(example=sample, return_answer=True)[-1200:])
print(type(format_example_without_batch(example=sample, return_answer=True)))
print(len(format_example_without_batch(example=sample, return_answer=True)))

<|im_start|>user
Carefully read and analyze the provided documents to answer the question that follows. Provide a detailed, step-by-step explanation of your reasoning, demonstrating how you arrived at your conclusion based on the information given in the documents.

<DOCUMENT>Statistical AnalysisThe two-way ANOVA (analysis of variance) was used for statistical analysis of the data generated in the experiments. For experiments involving two variables, where one was not statistically significant according to two-way ANOVA, the insignificant variable was eliminated. The data analysis was then repeated using one-way ANOVA. The data normality was verified with the Shapiro\u2013Wilk test and percentages were transformed into Arcsine. Means were compared using the Least Significant Difference (LSD) or Tukey\u2019s honestly significant difference (HSD). For statistical analysis, the data concerning gene expression were transformed into natural logarithms and the significance of the differences

In [ ]:
input_text = format_example_without_batch(example=sample, return_answer=True)
print(len(input_text))

20450


In [ ]:
input_text = format_example_without_batch(example=sample, return_answer=False)
input_ids = tokenizer(text=input_text, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, **generation_config)
print(tokenizer.batch_decode(outputs[:, input_ids["input_ids"].shape[1]:])[0])

<think>
Okay, let's tackle this question about the role of Aspergillus PCR in diagnosing invasive aspergillosis. First, I need to go through the provided documents to find relevant information. 

Looking at the third document, there's a section titled "4.1. Aspergillus" which discusses invasive aspergillosis (IA). The document mentions that Aspergillus PCR has been widely used in clinical settings for both screening and diagnosis, especially in high-risk patients like transplant recipients or those with neutropenia. It also notes that the FPCRI (Fungal PCR Initiative) has standardized methods for Aspergillus PCR, which has led to its inclusion in EORTC/MSGERC guidelines.

The text explains that PCR on blood samples (serum, plasma, whole blood) is effective, with specific sample volumes required. They mention that whole blood and plasma have higher sensitivity, while serum is more specific. There's a part about cycle threshold (Ct) values being important for interpreting results, and th

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 5120, padding_idx=151654)
        (layers): ModuleList(
          (0-5): 6 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from trl import SFTTrainer, SFTConfig

In [ ]:
args_definition = dict(
    dataset_text_field = "text",
    output_dir="/mistral-lora",
    overwrite_output_dir=True,
    eval_strategy="steps",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    max_steps=100,
    lr_scheduler_type="cosine",
    optim = "adamw_8bit",
    max_grad_norm = 0.3,
    warmup_steps=10,
    logging_steps=5,
    save_steps=5,
    logging_first_step=True,
    weight_decay = 0.01,
    seed=1399,
    bf16=True,
    report_to="wandb",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    packing=True,
    max_seq_length=max_seq_length,
)
args = SFTConfig(**args_definition)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/3148 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/350 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 64225280 || all params: 8627584000 || trainable%: 0.7444179042475855


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,148 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 64,225,280/14,000,000,000 (0.46% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: dogank01 (dogank01-turkish-german-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
5,1.820100,1.815350
10,1.809100,1.693627
15,1.684300,1.659180
20,1.676000,1.636236
25,1.617500,1.613292
30,1.637700,1.596650
35,1.598600,1.583212
40,1.564000,1.573269
45,1.552000,1.566148
50,1.626100,1.559840


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
def generate_responses(example, ft_model, og_model):
    input_text = format_example_without_batch(example=example, return_answer=False)
    input_ids_ft = tokenizer(text=input_text, return_tensors="pt").to(device)
    #input_ids_og = tokenizer(text=input_text, return_tensors="pt")
    ft_outputs = ft_model.generate(**input_ids_ft, **generation_config)
    #og_outputs = og_model.generate(**input_ids_og, **generation_config)

    display(Markdown("#### Question:"))
    display(Markdown(example["question"]))
    #display(Markdown("#### Original Completion:"))
    #display(Markdown(tokenizer.batch_decode(og_outputs[:, input_ids_og["input_ids"].shape[1]:], skip_special_tokens=True)[0]))

    display(Markdown("#### RAFT Fine-tuned Completion:"))
    display(Markdown(tokenizer.batch_decode(ft_outputs[:, input_ids_ft["input_ids"].shape[1]:], skip_special_tokens=False)[0]))

    display(Markdown("#### Expected Answer:"))
    display(Markdown(" {answer}".format(answer=example["cot_answer"])))
    display(Markdown("-----------------------------"))

In [ ]:
from IPython.display import Markdown, display
from tqdm import tqdm
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

# Load the Jina Embedding Model (once outside the function)
embedding_model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True)

In [ ]:
from tqdm import tqdm

In [ ]:
def evaluation(dataset, ft_model):
    """
    Evaluates the fine-tuned model on the given dataset.
    After each example, it prints the current average cosine similarity.

    Args:
        dataset: List of examples (each is a dict)
        ft_model: Fine-tuned model (HuggingFace Transformers model)
        tokenizer: Tokenizer used for input encoding
        generation_config: Dict of generation parameters
        device: "cuda" or "cpu"

    Returns:
        avg_score: Final average cosine similarity across all examples
        similarity_scores: List of individual scores
    """
    similarity_scores = []
    total_similarity = 0.0
    skipped_count = 0

    # Display progress bar
    for idx, example in enumerate(tqdm(dataset, desc="🧠 Evaluating Examples")):
        # Format input text
        input_text = format_example_without_batch(example=example, return_answer=False)

        # Tokenize input
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        # 🔍 Check if input is too long (more than 8192 tokens)
        if inputs["input_ids"].shape[1] > 8192:
            skipped_count += 1
            display(Markdown(f"### Example {idx + 1} ❌ Skipped"))
            display(Markdown(f"- **Input length:** `{inputs['input_ids'].shape[1]}` tokens"))
            display(Markdown("- **Reason:** Exceeds max allowed input length (8192 tokens)"))
            display(Markdown("---"))
            continue

        # Generate response
        with torch.no_grad():
            outputs = ft_model.generate(**inputs, **generation_config)

        # Decode generated response
        generated_text = tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True
        )

        # Get expected answer from dataset
        expected_text = example["cot_answer"]

        # Encode both texts using Jina Embeddings v3
        try:
            gen_emb = embedding_model.encode([generated_text], task="text-matching")
            exp_emb = embedding_model.encode([expected_text], task="text-matching")

            # Convert to tensors
            gen_tensor = torch.tensor(gen_emb).to(device)
            exp_tensor = torch.tensor(exp_emb).to(device)

            # Compute cosine similarity
            similarity = F.cosine_similarity(gen_tensor, exp_tensor, dim=1).item()
            similarity_scores.append(similarity)
            total_similarity += similarity

            # Compute running average
            running_avg = total_similarity / len(similarity_scores)

        except Exception as e:
            print(f"⚠️ Error processing example: {e}")
            continue

        # Optional: Print detailed results for each example
        display(Markdown(f"### Example {idx + 1} ✅ Processed"))
        display(Markdown("**#### Question:**"))
        display(Markdown(example["question"]))

        display(Markdown("**#### Generated Answer:**"))
        display(Markdown(generated_text))

        display(Markdown("**#### Expected Answer:**"))
        display(Markdown(expected_text))

        display(Markdown(f"**#### Cosine Similarity:** `{similarity:.4f}`"))
        display(Markdown(f"**📈 Running Average So Far:** `{running_avg:.4f}`"))
        display(Markdown("---"))

    # Final average
    final_avg = total_similarity / len(similarity_scores) if similarity_scores else 0
    display(Markdown("## ✅ Final Summary"))
    display(Markdown(f"- **Total Examples Evaluated:** `{len(similarity_scores)}`"))
    display(Markdown(f"- **Examples Skipped (too long):** `{skipped_count}`"))
    display(Markdown(f"- **Final Average Cosine Similarity:** `{final_avg:.4f}`"))

In [ ]:
evaluation(
    dataset=val_data,
    ft_model=model,
)

🧠 Evaluating Examples:   0%|          | 0/350 [00:00<?, ?it/s]

### Example 1 ✅ Processed

**#### Question:**

What does Table S1 provide estimates for?

**#### Generated Answer:**

...

🧠 Evaluating Examples: 100%|██████████| 350/350 [1:21:04<00:00, 13.90s/it]


## ✅ Final Summary

- **Total Examples Evaluated:** `186`

- **Examples Skipped (too long):** `164`

- **Final Average Cosine Similarity:** `0.8882`

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display
from ipywidgets import interact, IntSlider


In [ ]:
model_save_name = "Qwen3-14B-unsloth-bnb-4bit-raft-ft"

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# Save model & tokenizer
model.push_to_hub(model_save_name)
tokenizer.push_to_hub(model_save_name)

README.md:   0%|          | 0.00/589 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/257M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Qwen3-14B-unsloth-bnb-4bit-raft-ft


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
# Save adapters
trainer.push_to_hub(model_save_name + "adapters")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/257M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DoganK01/mistral-lora/commit/395cf6a4ea42eb96a734477b08b1f3705304e0a9', commit_message='Qwen3-14B-unsloth-bnb-4bit-raft-ftadapters', commit_description='', oid='395cf6a4ea42eb96a734477b08b1f3705304e0a9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DoganK01/mistral-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='DoganK01/mistral-lora'), pr_revision=None, pr_num=None)